In [1]:
import pandas as pd
import numpy as np
import os

# Read raw population data

In [2]:
def prepare_total_pop():
    total_pop = pd.read_csv("../data/raw/WPP2019_TotalPopulationBySex.csv", sep=",", na_values='')
    total_pop = total_pop[total_pop["Time"] < 2020]
    total_pop.drop(columns=["LocID", "VarID", "Variant", "MidPeriod"], inplace=True)

    total_pop.columns = ["Country", "Year", "PopMale", "PopFemale", "PopTotal"]
    total_pop.set_index(["Country", "Year"], inplace=True)

    total_pop[["PopMale", "PopFemale", "PopTotal"]] = (total_pop[["PopMale", "PopFemale", "PopTotal"]].fillna(0) * 1000).astype("int64")
    return total_pop

def prepare_age_pop():
    age_pop = pd.read_csv("../data/raw/WPP2019_PopulationByAgeSex_Medium.csv", sep=",", na_values='')
    age_pop = age_pop[age_pop["Time"] < 2020]
    age_pop.drop(columns=["LocID", "VarID", "Variant", "MidPeriod", "AgeGrpStart", "AgeGrpSpan"], inplace=True)

    age_pop.columns = ["Country", "Year", "AgeGrp", "PopMale", "PopFemale", "PopTotal"]
    age_pop.set_index(["Country", "Year"], inplace=True)

    age_pop[["PopMale", "PopFemale", "PopTotal"]] = (age_pop[["PopMale", "PopFemale", "PopTotal"]].fillna(0) * 1000).astype("int64")
    return age_pop

In [3]:
pop_total = prepare_total_pop()
pop_per_age = prepare_age_pop()

In [4]:
pop_total.head(2)

PopMale  PopFemale  PopTotal
Country     Year                              
Afghanistan 1950  4099242    3652874   7752116
            1951  4134755    3705395   7840151

In [5]:
pop_per_age.head(2)

AgeGrp  PopMale  PopFemale  PopTotal
Country     Year                                     
Afghanistan 1950    0-4   630044     661578   1291622
            1950    5-9   516206     487335   1003541

# Rearrange Population data

## Aggregate the age groups in three larger groups, from 0-19, 20-59, and 60+ years

In [6]:
pop_per_age_young = pop_per_age.query('AgeGrp in ["0-4", "5-9", "10-14", "15-19"]').sum(level=['Country', 'Year'])
pop_per_age_young.rename(columns={"PopMale": "PopMale_0-19", "PopFemale": "PopFemale_0-19", "PopTotal" : "PopTotal_0-19"}, inplace=True)
pop_per_age_mid = pop_per_age.query('AgeGrp in ["20-24", "25-29", "30-34", "35-39", "40-44", "45-49","50-54", "55-59"]').sum(level=['Country', 'Year'])
pop_per_age_mid.rename(columns={"PopMale": "PopMale_20-59", "PopFemale": "PopFemale_20-59", "PopTotal" : "PopTotal_20-59"}, inplace=True)
pop_per_age_old = pop_per_age.query('AgeGrp in ["60-64", "65-69", "70-74", "75-79", "80-84", "85-89", "90-94", "95-99", "100+"]').sum(level=['Country', 'Year'])
pop_per_age_old.rename(columns={"PopMale": "PopMale_60+", "PopFemale": "PopFemale_60+", "PopTotal" : "PopTotal_60+"}, inplace=True)

## Merge age groups with total population

In [7]:
pop_with_groups = pop_per_age_young.merge(pop_per_age_mid, left_index=True, right_index=True)
pop_with_groups = pop_with_groups.merge(pop_per_age_old, left_index=True, right_index=True)
pop_total_with_groups = pop_total.merge(pop_with_groups, left_index=True, right_index=True)

# Load indicators dataset

In [8]:
indicators = pd.read_csv("../data/raw/WPP2019_Period_Indicators_Medium.csv", sep=",", na_values='')
indicators[["Births", "Deaths", "DeathsMale", "DeathsFemale", "NetMigrations"]] = (indicators[["Births", "Deaths", "DeathsMale", "DeathsFemale", "NetMigrations"]].fillna(0) * 1000 / 5).astype("int64")
indicators_pop = indicators.merge(pop_total_with_groups, left_on=['MidPeriod','Location'], right_on=['Year', 'Country'])
indicators_pop["RelMigrations"] = indicators_pop["NetMigrations"] / indicators_pop["PopTotal"]
indicators_pop.drop(columns=['VarID', 'Variant'], inplace=True)
indicators_pop.rename(columns={"Location" : "Country"}, inplace=True)

## Scale population columns by total population

In [9]:
scale_cols = ["Births", "Deaths", "DeathsMale", "DeathsFemale", "PopMale", "PopFemale", "PopMale_0-19", "PopFemale_0-19",
              "PopTotal_0-19", "PopMale_20-59", "PopFemale_20-59", "PopTotal_20-59", "PopMale_60+", "PopFemale_60+", "PopTotal_60+"]
indicators_pop[scale_cols] = indicators_pop[scale_cols].div(indicators_pop["PopTotal"], axis=0)

# Read fragile states data
## Merge it with `indicators_pop`

In [10]:
fragile_states = pd.read_csv("../data/clean/fragile_states_index.csv", sep=",", na_values='')
fragile_states["country"] = fragile_states["country"].str.strip()
full_set = indicators_pop.merge(fragile_states, left_on=['Country','MidPeriod'], right_on=['country','year'])

## Concat not merged countries manually

In [11]:
missing_countries = list(set(fragile_states["country"]).difference(set(full_set["country"])))
missing_countries.remove("Eswatini")
missing_countries.remove("Czechia")
missing_countries.sort()
corresponding_countries = ["Bolivia (Plurinational State of)", "Cabo Verde", "Democratic Republic of the Congo", "Congo", "Côte d'Ivoire", "Czechia", "Guinea-Bissau", "Iran (Islamic Republic of)", "Israel", "Kyrgyzstan", "Lao People's Democratic Republic", "North Macedonia", "Republic of Moldova", "Dem. People's Republic of Korea", "Russian Federation", "Slovakia", "Republic of Korea", "Eswatini", "Syrian Arab Republic", "United Republic of Tanzania", "United States of America", "Venezuela (Bolivarian Republic of)", "Viet Nam"]
replacements = list(zip(missing_countries, corresponding_countries))
replacements

[('Bolivia', 'Bolivia (Plurinational State of)'),
 ('Cape Verde', 'Cabo Verde'),
 ('Congo Democratic Republic', 'Democratic Republic of the Congo'),
 ('Congo Republic', 'Congo'),
 ("Cote d'Ivoire", "Côte d'Ivoire"),
 ('Czech Republic', 'Czechia'),
 ('Guinea Bissau', 'Guinea-Bissau'),
 ('Iran', 'Iran (Islamic Republic of)'),
 ('Israel and West Bank', 'Israel'),
 ('Kyrgyz Republic', 'Kyrgyzstan'),
 ('Laos', "Lao People's Democratic Republic"),
 ('Macedonia', 'North Macedonia'),
 ('Moldova', 'Republic of Moldova'),
 ('North Korea', "Dem. People's Republic of Korea"),
 ('Russia', 'Russian Federation'),
 ('Slovak Republic', 'Slovakia'),
 ('South Korea', 'Republic of Korea'),
 ('Swaziland', 'Eswatini'),
 ('Syria', 'Syrian Arab Republic'),
 ('Tanzania', 'United Republic of Tanzania'),
 ('United States', 'United States of America'),
 ('Venezuela', 'Venezuela (Bolivarian Republic of)'),
 ('Vietnam', 'Viet Nam')]

In [12]:
for old, new in replacements:
    fragile_states["country"] = fragile_states["country"].str.replace(pat=old, repl=new)
fragile_states["country"].unique()

array(['Sudan', 'Democratic Republic of the Congo', "Côte d'Ivoire",
       'Iraq', 'Zimbabwe', 'Chad', 'Somalia', 'Haiti', 'Pakistan',
       'Afghanistan', 'Guinea', 'Liberia', 'Central African Republic',
       "Dem. People's Republic of Korea", 'Burundi', 'Sierra Leone',
       'Yemen', 'Myanmar', 'Bangladesh', 'Nepal', 'Uganda', 'Nigeria',
       'Uzbekistan', 'Rwanda', 'Sri Lanka', 'Ethiopia', 'Colombia',
       'Kyrgyzstan', 'Malawi', 'Burkina Faso', 'Egypt', 'Indonesia',
       'Kenya', 'Syrian Arab Republic', 'Bosnia and Herzegovina',
       'Cameroon', 'Angola', 'Togo', 'Bhutan',
       "Lao People's Democratic Republic", 'Mauritania', 'Tajikistan',
       'Russian Federation', 'Niger', 'Turkmenistan', 'Guinea-Bissau',
       'Cambodia', 'Dominican Republic', 'Papua New Guinea', 'Belarus',
       'Guatemala', 'Equatorial Guinea', 'Iran (Islamic Republic of)',
       'Eritrea', 'Serbia', 'Bolivia (Plurinational State of)', 'China',
       'Republic of Moldova', 'Nicaragua', 'G

## Merge again

In [13]:
full_set = indicators_pop.merge(fragile_states, left_on=['Country','MidPeriod'], right_on=['country','year'])

In [14]:
list(set(fragile_states["country"]).difference(set(full_set["country"])))

[]

## Drop unused/duplicated columns

In [15]:
indicators_pop.drop(columns=["LocID"], inplace=True)
full_set.drop(columns=["LocID", "Unnamed: 0"], inplace=True)

## Calculate the index change from the previous 5 year

In [17]:
query_result = full_set.query("Time == '2005-2010'")
full_set.loc[query_result.index, "change_from_previous_year"] = pd.Series(0, index=query_result.index)
previous = query_result
for years in ["2010-2015", "2015-2020"]:
    query_result = full_set.query("Time == '" + years + "'")
    full_set.loc[query_result.index, "change_from_previous_year"] = \
        (query_result.set_index("Country")["total"] - previous.set_index("Country")["total"]).round(1).fillna(0).to_numpy()
    previous = full_set.loc[query_result.index, :]

# Save all dataframes

In [19]:
pop_total.to_csv("../data/clean/population_total.csv")
pop_per_age.to_csv("../data/clean/population_per_age.csv")
indicators_pop.to_csv(index=False, path_or_buf="../data/clean/population_indicators.csv")
full_set.to_csv(index=False, path_or_buf="../data/clean/full_set.csv")